Using Keras with Scikit-learn and using grid search for hyperparameters.

In [22]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.max_columns',1000)
print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pd.__version__))
print('Numpy: {}'.format(np.__version__))

Python: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
Pandas: 0.23.0
Numpy: 1.19.1


In [23]:
#Read and load the data
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
file = 'diabetes.csv'
data = pd.read_csv('diabetes.csv')
print(data.describe())
data.info()

       Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
count   768.000000  768.000000     768.000000     768.000000  768.000000   
mean      3.845052  120.894531      69.105469      20.536458   79.799479   
std       3.369578   31.972618      19.355807      15.952218  115.244002   
min       0.000000    0.000000       0.000000       0.000000    0.000000   
25%       1.000000   99.000000      62.000000       0.000000    0.000000   
50%       3.000000  117.000000      72.000000      23.000000   30.500000   
75%       6.000000  140.250000      80.000000      32.000000  127.250000   
max      17.000000  199.000000     122.000000      99.000000  846.000000   

              BMI  DiabetesPedigreeFunction         Age     Outcome  
count  768.000000                768.000000  768.000000  768.000000  
mean    31.992578                  0.471876   33.240885    0.348958  
std      7.884160                  0.331329   11.760232    0.476951  
min      0.000000                  

In [25]:
#Preprocess the data
from sklearn.preprocessing import StandardScaler
print(data[data['Glucose'] == 0])
columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in columns:
    data[col].replace(0, np.NaN, inplace=True)
print(data.describe())
data.dropna(inplace = True)
print("\n")
data.describe()

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
75             1        0             48             20        0  24.7   
182            1        0             74             20       23  27.7   
342            1        0             68             35        0  32.0   
349            5        0             80             32        0  41.0   
502            6        0             68             41        0  39.0   

     DiabetesPedigreeFunction  Age  Outcome  
75                      0.140   22        0  
182                     0.299   21        0  
342                     0.389   22        0  
349                     0.346   37        1  
502                     0.727   41        1  
       Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
count   768.000000  763.000000     733.000000     541.000000  394.000000   
mean      3.845052  121.686763      72.405184      29.153420  155.548223   
std       3.369578   30.535641      12.382158      

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [28]:
#Convert to numpy arrays
dataset = data.values
print(dataset.shape)
print(data.head(n = 5))
X = dataset[:,0:8]
Y = dataset[:,8].astype(int)
print(X.shape)
print(Y.shape)
print(Y[:5])

(392, 9)
    Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
3             1     89.0           66.0           23.0     94.0  28.1   
4             0    137.0           40.0           35.0    168.0  43.1   
6             3     78.0           50.0           32.0     88.0  31.0   
8             2    197.0           70.0           45.0    543.0  30.5   
13            1    189.0           60.0           23.0    846.0  30.1   

    DiabetesPedigreeFunction  Age  Outcome  
3                      0.167   21        0  
4                      2.288   33        1  
6                      0.248   26        1  
8                      0.158   53        1  
13                     0.398   59        1  
(392, 8)
(392,)
[0 1 1 1 1]


In [29]:
scaler = StandardScaler().fit(X)
X_standardized = scaler.transform(X)
data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [30]:
#Let's import the necessary models/functions
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [31]:
#Define the model 
def getModel():
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model
model = getModel()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
#Grid search for the optimal number of parameters and epochs
seed = 6
np.random.seed(seed)
model = KerasClassifier(build_fn = getModel, verbose = 1)
batch_size = [10, 20, 40]
epochs = [10, 50, 100]
#Make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)
# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

#Summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=10, epochs=10 ........................................
Epoch 1/10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


32/32 [==============================] - 0s 1ms/step - loss: 0.5759 - accuracy: 0.7061
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4586 - accuracy: 0.7508
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4351 - accuracy: 0.7796
Epoch 4/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4284 - accuracy: 0.8019
Epoch 5/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4205 - accuracy: 0.7955
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 0.4168 - accuracy: 0.8147
Epoch 7/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4072 - accuracy: 0.7955
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 0.3954 - accuracy: 0.8115
Epoch 9/10
32/32 [==============================] - 0s 1ms/step - loss: 0.3932 - accuracy: 0.8179
Epoch 10/10
8/8 [==============================] - 0s 1ms/step - loss: 0.5335 - accuracy: 0.7595
[CV] ............ batch_size=10,

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


32/32 [==============================] - 0s 2ms/step - loss: 0.6589 - accuracy: 0.7412
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 0.5753 - accuracy: 0.8019
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 0.5312 - accuracy: 0.8275
Epoch 4/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4970 - accuracy: 0.8275
Epoch 5/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4772 - accuracy: 0.8211
Epoch 6/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4628 - accuracy: 0.8243
Epoch 7/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4479 - accuracy: 0.8371
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4376 - accuracy: 0.8339
Epoch 9/10
32/32 [==============================] - 0s 1ms/step - loss: 0.4317 - accuracy: 0.8307
Epoch 10/10
8/8 [==============================] - 0s 887us/step - loss: 0.7801 - accuracy: 0.6076
[CV] ............ batch_size=1

...

8/8 [==============================] - 0s 1ms/step - loss: 0.6736 - accuracy: 0.6497
Epoch 2/100
8/8 [==============================] - 0s 2ms/step - loss: 0.6107 - accuracy: 0.6656
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: 0.5298 - accuracy: 0.6656
Epoch 4/100
8/8 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.6656
Epoch 5/100
8/8 [==============================] - 0s 2ms/step - loss: 0.4942 - accuracy: 0.6656
Epoch 6/100
8/8 [==============================] - 0s 2ms/step - loss: 0.4757 - accuracy: 0.7420
Epoch 7/100
8/8 [==============================] - 0s 2ms/step - loss: 0.4617 - accuracy: 0.7771
Epoch 8/100
8/8 [==============================] - 0s 2ms/step - loss: 0.4501 - accuracy: 0.7771
Epoch 9/100
8/8 [==============================] - 0s 2ms/step - loss: 0.4470 - accuracy: 0.7834
Epoch 10/100
8/8 [==============================] - 0s 1ms/step - loss: 0.4429 - accuracy: 0.7834
Epoch 11/100
8/8 [=======================

8/8 [==============================] - 0s 1ms/step - loss: 0.3119 - accuracy: 0.8758
Epoch 85/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3081 - accuracy: 0.8631
Epoch 86/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3086 - accuracy: 0.8599
Epoch 87/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3063 - accuracy: 0.8726
Epoch 88/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3052 - accuracy: 0.8790
Epoch 89/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3019 - accuracy: 0.8758
Epoch 90/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3060 - accuracy: 0.8790
Epoch 91/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3031 - accuracy: 0.8790
Epoch 92/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3006 - accuracy: 0.8790
Epoch 93/100
8/8 [==============================] - 0s 1ms/step - loss: 0.3040 - accuracy: 0.8790
Epoch 94/100
8/8 [===============

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.8min finished


10/10 [==============================] - 0s 1ms/step - loss: 0.6659 - accuracy: 0.6582
Epoch 2/100
10/10 [==============================] - 0s 1ms/step - loss: 0.5659 - accuracy: 0.7219
Epoch 3/100
10/10 [==============================] - 0s 1ms/step - loss: 0.4856 - accuracy: 0.7730
Epoch 4/100
10/10 [==============================] - 0s 1ms/step - loss: 0.4580 - accuracy: 0.7628
Epoch 5/100
10/10 [==============================] - 0s 1ms/step - loss: 0.4433 - accuracy: 0.7730
Epoch 6/100
10/10 [==============================] - 0s 2ms/step - loss: 0.4409 - accuracy: 0.7883
Epoch 7/100
10/10 [==============================] - 0s 1ms/step - loss: 0.4328 - accuracy: 0.7883
Epoch 8/100
10/10 [==============================] - 0s 1ms/step - loss: 0.4286 - accuracy: 0.7857
Epoch 9/100
10/10 [==============================] - 0s 1ms/step - loss: 0.4254 - accuracy: 0.7908
Epoch 10/100
10/10 [==============================] - 0s 1ms/step - loss: 0.4221 - accuracy: 0.7908
Epoch 11/100
10/10 [=

10/10 [==============================] - 0s 1ms/step - loss: 0.2993 - accuracy: 0.8699
Epoch 84/100
10/10 [==============================] - 0s 2ms/step - loss: 0.3035 - accuracy: 0.8648
Epoch 85/100
10/10 [==============================] - 0s 1ms/step - loss: 0.3043 - accuracy: 0.8699
Epoch 86/100
10/10 [==============================] - 0s 1ms/step - loss: 0.3099 - accuracy: 0.8699
Epoch 87/100
10/10 [==============================] - 0s 1ms/step - loss: 0.3006 - accuracy: 0.8776
Epoch 88/100
10/10 [==============================] - 0s 1ms/step - loss: 0.2961 - accuracy: 0.8750
Epoch 89/100
10/10 [==============================] - 0s 1ms/step - loss: 0.3035 - accuracy: 0.8724
Epoch 90/100
10/10 [==============================] - 0s 1ms/step - loss: 0.2898 - accuracy: 0.8750
Epoch 91/100
10/10 [==============================] - 0s 1ms/step - loss: 0.2917 - accuracy: 0.8724
Epoch 92/100
10/10 [==============================] - 0s 1ms/step - loss: 0.2888 - accuracy: 0.8750
Epoch 93/100


In [39]:
#Next, do a grid search for the learning rate and the Dropout
from keras.layers import Dropout
seed = 6
np.random.seed(seed)
def createModel(learn_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model
# create the model
model = KerasClassifier(build_fn = createModel, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))
    

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.772, total=   2.3s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.608, total=   2.4s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


...

[CV] ... dropout_rate=0.0, learn_rate=0.01, score=0.808, total=   2.1s
[CV] dropout_rate=0.0, learn_rate=0.01 ...............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   19.3s remaining:    0.0s


[CV] ... dropout_rate=0.0, learn_rate=0.01, score=0.859, total=   2.5s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.684, total=   2.1s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.595, total=   2.0s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.782, total=   2.0s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.782, total=   2.1s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.808, total=   2.0s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.772, total=   2.3s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.6min finished


Best: 0.7836091995239258, using {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7708211541175842 (0.08502745175936657) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7836091995239258 (0.09265233572904454) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7300551652908325 (0.07981850216877587) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.7580331087112426 (0.08388572279933751) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.780947744846344 (0.0658331840038728) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7401492953300476 (0.07664480638961145) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.7759493589401245 (0.08411546524215682) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7758844494819641 (0.06678233097270206) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.7376825690269471 (0.09058435789301031) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


In [40]:
#Generate predictions with the optional hyperparameters and the confusion matrix
from sklearn.metrics import classification_report, accuracy_score
y_pred = grid.predict(X_standardized)
print(y_pred.shape)
print(y_pred[:5])
print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
(392, 1)
[[0]
 [1]
 [0]
 [1]
 [1]]
0.8724489795918368
              precision    recall  f1-score   support

           0       0.91      0.90      0.90       262
           1       0.80      0.82      0.81       130

    accuracy                           0.87       392
   macro avg       0.86      0.86      0.86       392
weighted avg       0.87      0.87      0.87       392

